<a href="https://colab.research.google.com/github/MOOwuttichai/Cancer_Social_Listening/blob/main/%E0%B8%9A%E0%B8%AD%E0%B8%81%E0%B9%82%E0%B8%A3%E0%B8%84%2B%E0%B9%80%E0%B8%9E%E0%B8%A8%2B%E0%B9%83%E0%B8%84%E0%B8%A3%E0%B9%80%E0%B8%9B%E0%B9%87%E0%B8%99%E0%B8%84%E0%B8%99%E0%B9%80%E0%B8%A5%E0%B9%88%E0%B8%B2%2BSentenceTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
!pip install attacut
!pip install pythainlp
!pip install epitran
!pip install -U sentence-transformers

In [54]:
import pandas as pd
from google.colab import drive
from attacut import tokenize, Tokenizer
import pythainlp
from pythainlp import sent_tokenize, word_tokenize, Tokenizer
from pythainlp.util import normalize
from pythainlp.corpus.common import thai_words
from collections import OrderedDict
import matplotlib as mpl
import numpy as np
from sklearn.cluster import KMeans
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [55]:
data = pd.read_csv('/content/drive/MyDrive/Carbon/data/ดึงข้อมูลFacebook 2 - Sheet1.csv')

In [56]:
comment=data.groupby('ชื่อ').sum().reset_index()

In [57]:
words = set(thai_words())
words.remove("โรคมะเร็ง")
name =['กระเพาะปัสสวะ','กระเพาะปัสสาวะ','เยื่อบุโพรงมดลูก','ปากมดลูก','เม็ดเลือดขาว','กระเพาะอาหาร','กระเพราะอาหาร','ต่อมไทรอยด์','ต่อมไทยรอยด์','ท่อน้ำดี']
for i in name:
  words.add(i)
custom_tokenizer = Tokenizer(words)

In [58]:
comment['คำพูดโรค'][0]

'วิธีตรวจมะเร็ง ได้ที่ไหนบ้างครับนอกจาก รพ เจาะเลือดไปตรวจได้ไหมครับ'

In [59]:
name_cancar = ['มะเร็งกระเพาะปัสสวะ','มะเร็งเยื่อบุโพรงมดลูก','มะเร็งปากมดลูก','มะเร็งเม็ดเลือดขาว','มะเร็งตับ','มะเร็งตับอ่อน','มะเร็งต่อมลูกหมาก','มะเร็งกระเพาะปัสสาวะ','มะเร็งกระเพาะอาหาร'
,'มะเร็งหลอดอาหาร','มะเร็งเต้านม','มะเร็งปอด','มะเร็งลำไส้ใหญ่','มะเร็งรังไข่','มะเร็งช่องปาก','มะเร็งต่อมไทรอยด์','มะเร็งต่อมน้ำเหลือง','มะเร็งสมอง','มะเร็งผิวหนัง','มะเร็งลำไส้'
,'มะเร็งท่อน้ำดี',"มะเร็งลำไส้ใหญ่และทวารหนัก","มะเร็งผิวหนังชนิดเมลาโนมา","มะเร็งต่อมน้ำเหลืองชนิดนอนฮอดจ์กิน","มะเร็งเม็ดเลือดขาวชนิดเฉียบพลัน","มะเร็งกระดูก","มะเร็งสมองและระบบประสาทกลาง","มะเร็งไต",
"มะเร็งมดลูก","มะเร็งถุงน้ำดี","มะเร็งกระดูกสันหลัง","มะเร็งลูกตา","มะเร็งเยื่อบุช่องคลอด","มะเร็งเยื่อบุอวัยวะเพศชาย","มะเร็งเยื่อบุอวัยวะเพศหญิง","มะเร็งต่อมหมวกไต","มะเร็งต่อมน้ำลาย","มะเร็งถุงน้ำรังไข่"
]#อันนี้ prom ของ gimini "จินี ช่วยสร้างlist python โดยในlistนั้นมีการระบุโรคมะเร็งชนิดต่างๆให้หน่อยขอเป็นเเบบภาษาทางการเเละก็ภาษาพูด ขอเเบบไม่ซ้ำด้วย"

In [60]:
new_colcan = []
for i in range(len(comment)):#len(comment)
  text= comment['คำพูดโรค'][i]
  custom_tokenizer = Tokenizer(words)
  Token = custom_tokenizer.word_tokenize(normalize(text))
  Token.append('end')
  #หาโรค
  #--------------------------------------------------------
  list_cancer = []
  for i in range(len(Token)):
    if (Token[i] == "มะเร็ง")|(Token[i] == "โรคมะเร็ง"):
      list_cancer.append(Token[i]+Token[i+1])
    # else :
    #   list_cancer.append('ไม่สามารถระบุได้')
  unique_list = list(OrderedDict.fromkeys(list_cancer))
  #----------------------------------------------------------
  list_define_cancer = []
  new_list=[]
  for i in range(len(unique_list)):
    for j in range(len(name_cancar)):
      if unique_list[i]==name_cancar[j]:
        list_define_cancer.append(unique_list[i])
  #-----------------------------------------------------
  cancer_list_de2 =[]
  if len(list_define_cancer) > 0:
    if len(list_define_cancer) == 2:
      cancer_list_de2.append('เล่ามากกว่า 2 โรค')
    elif len(list_define_cancer)==1:
      cancer_list_de2.append(list_define_cancer[0])
  elif list_define_cancer==[]:
    cancer_list_de2.append('ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น')
  new_colcan.append(cancer_list_de2[0])
print(new_colcan)

['ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'มะเร็งปากมดลูก', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'มะเร็งเต้านม', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'มะเร็งรังไข่', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่

In [69]:
Genden = {'ชาย':['พ่อ','บิดา','พี่ชาย','น้องชาย','ลูกชาย','สามี','พัว','ผัว','ปู่','ตา','คุณปู่','คุณตา','คุณพ่อ',
                 'ปู่ทวด','ตาทวด','ลุง','อาหนู','คุณอา','คุณลุง','หลายชาย','ลูกเขย','เขย','พี่เขย','น้องเขย',
                 'พ่อตา','พ่อผม','พ่อหนู','พ่อพม','ชาย','หนุ่ม','ลช.','ผ่อ','ชย.','น้าชาย','ผ่อตา','หน.']
          ,'หญิง':['แม่','เเม่','คุณแม่','มารดา','พี่สาว','น้องสาว','ลูกสาว','ภรรยา','เมีย','ย่า','ยาย','คุณย่า',
                   'คุณยาย','คุณเเม่','ย่าทวด','ยายทวด','ป้า','น้า','คุณป้า','คุณน้า','หลายสาว','ลูกสะใถ้',
                   'ลูกสะใภ้','สะใภ้','พี่สะใภ้','น้องสะใภ้','เเม่ผม','เเม่หนู','เเม่พม','แม่ผม','แม่หนู','แม่พม','สาว','หญิง','ก้อน','คลำ']}

In [70]:
len(Genden['ชาย'])

36

In [71]:
new_colgenden=[]
for i in range(len(comment)):
  text= comment['คำพูดโรค'][i]
  custom_tokenizer = Tokenizer(words)
  Token = custom_tokenizer.word_tokenize(normalize(text))
  Token.append('end')
  list_genden = []
  for j in range(len(Genden['หญิง'])):
    for k in range(len(Token)):
      if (Token[k] == Genden['ชาย'][j]):
        list_genden.append('เพศชาย')
      elif(Token[k] == Genden['หญิง'][j]):
        list_genden.append('เพศหญิง')
  genden_list =[]
  genden_list = list(OrderedDict.fromkeys(list_genden)) # ลบคำซ้ำ
  #-------------------------------------------------------------------
  list_define_genden = []
  if len(genden_list) > 0 :
    if len(genden_list) == 2:
      list_define_genden.append('เล่าทั้งสองเพศ')
    elif len(genden_list)==1:
      list_define_genden.append(genden_list[0])
  elif len(genden_list)==0:
    list_define_genden.append('ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น')
  genden_list_de =[]
  genden_list_de = list(OrderedDict.fromkeys(list_define_genden))
  new_colgenden.append(genden_list_de[0])
print(new_colgenden)

['ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'เพศหญิง', 'เพศหญิง', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'เพศหญิง', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'เพศหญิง', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'เพศหญิง', 'เพศหญิง', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'เพศหญิง', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น

In [72]:
comment

,ชื่อ,คำพูดโรค
0,Akekapong Bowling Singharas,วิธีตรวจมะเร็ง ได้ที่ไหนบ้างครับนอกจาก รพ เจาะ...
1,Aliis Chattarika,Theboom Moungna
2,Alisa Chuasa,Jurairat Norasri. พงษ์นิวัฒน์ ชูอาษา
3,Amornrat Pollamuangdee,คลำเจอก้อนที่เต้านมค่ะ
4,Anothai Hassabut,คัดกรองเบื้องต้นด้วยการตรวจเต้านมด้วยตัวเองว่า...
...,...,...
276,แม่รุ้ง น้องเกล,ลูกสาวมีอาการปวดท้องแบบแน่นๆ เวียนหัว มีจุดจ้ำ...
277,แอนนี่ น้องข้าวโอ๊ต,Wanitchaya Puruean
278,แอม' มม มมมมมมม.,มีก้อนเนื้อที่เต้านม ไม่มีอาการเจ็บ แต่คลำๆแล้...
279,โบว์ คะ,จับนมแล้วเจอก้อน


In [73]:
point =[]
k =[]
r=comment
for i in range(len(comment)):
  text= comment['คำพูดโรค'][i]
  custom_tokenizer = Tokenizer(words)
  Token = custom_tokenizer.word_tokenize(normalize(text))
  Token.append('end')
  count_text = len(Token)-1
  if new_colcan[i] =='ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น':
    x=0
  elif new_colcan[i] == 'เล่ามากกว่า 2 โรค':
    x=1
  else :
    x=2
  point.append(count_text+x)
  pop = point[i]/100
  if pop >= 0.75:
    k.append('มีประโยชน์มาก')
  elif pop < 0.75 and pop >=0.5:
    k.append('มีประโยชน์')
  elif pop < 0.5 and pop >=0.25:
    k.append('อาจมีประโยชน์')
  elif pop < 0.25 :
    k.append('ไม่มีประโยชน์')
# elif max(point) > 100:
#   for i in range(len(comment)):
#     pop = point[i]/max(point)
#     if pop >= 0.75:
#       k.append('มีประโยชน์มาก')
#     elif pop < 0.75 and pop >=0.5:
#       k.append('มีประโยชน์')
#     elif pop < 0.5 and pop >=0.25:
#       k.append('อาจมีประโยชน์')
#     elif pop < 0.25 :
#       k.append('ไม่มีประโยชน์')
r['โรค'] = new_colcan
r['คะเเนน'] = point
r['ผลลัพธ์'] = k
print(r)

                            ชื่อ  \
0    Akekapong Bowling Singharas   
1               Aliis Chattarika   
2                   Alisa Chuasa   
3         Amornrat Pollamuangdee   
4               Anothai Hassabut   
..                           ...   
276              แม่รุ้ง น้องเกล   
277          แอนนี่ น้องข้าวโอ๊ต   
278             แอม' มม มมมมมมม.   
279                      โบว์ คะ   
280   โรคร้ายผ่านได้ถ้าไม่ยอมแพ้   

                                              คำพูดโรค  \
0    วิธีตรวจมะเร็ง ได้ที่ไหนบ้างครับนอกจาก รพ เจาะ...   
1                                      Theboom Moungna   
2                 Jurairat Norasri. พงษ์นิวัฒน์ ชูอาษา   
3                               คลำเจอก้อนที่เต้านมค่ะ   
4    คัดกรองเบื้องต้นด้วยการตรวจเต้านมด้วยตัวเองว่า...   
..                                                 ...   
276  ลูกสาวมีอาการปวดท้องแบบแน่นๆ เวียนหัว มีจุดจ้ำ...   
277                                 Wanitchaya Puruean   
278  มีก้อนเนื้อที่เต้านม ไม่มีอาการเจ็บ แต่ค

In [74]:
def detect_person(comment):
    # คำที่ใช้ตรวจสอบว่ามีใครเป็นคนอยู่ในความคิดเห็น
    other = ['พ่อ','บิดา','พี่ชาย','น้องชาย','ลูกชาย','สามี','พัว','ผัว','ปู่','ตา','คุณปู่','คุณตา','คุณพ่อ',
              'ปู่ทวด','ตาทวด','ลุง','อาหนู','คุณอา','คุณลุง','หลายชาย','ลูกเขย','เขย','พี่เขย','น้องเขย',
              'พ่อตา','พ่อผม','พ่อหนู','พ่อพม','ชาย','หนุ่ม''แม่','เเม่','คุณแม่','มารดา','พี่สาว','น้องสาว',
              'ลูกสาว','ภรรยา','เมีย','ย่า','ยาย','คุณย่า','คุณยาย','คุณเเม่','ย่าทวด','ยายทวด','ป้า',
              'น้า','คุณป้า','คุณน้า','หลายสาว','ลูกสะใถ้','ลูกสะใภ้','สะใภ้','พี่สะใภ้','น้องสะใภ้','เเม่ผม','เเม่หนู','เเม่พม','แม่ผม','แม่หนู','แม่พม','สาว','หญิง']
    myself = ['ผมเป็น','ผมเอง','กระผม','พมเป็น','พมเอง','กระพม','หนูเอง','หนู','ดิฉัน','ตัวเอง','ก้อน','คลำ']
    # ตรวจสอบคำในความคิดเห็น
    for keyword in  other:
        if keyword in comment and "อาการ" not in comment :
              return 'เล่าประสบการณ์คนอื่น'
    for keyword in  myself:
        if keyword in comment:
            return 'เล่าประสบการณ์ตัวเอง'
    # หากไม่พบคำที่บ่งบอกถึงคน
    return 'ไม่ได้เล่าประสบการณ์'

In [75]:
def detect_gender_other(comment):
    # คำที่บ่งบอกถึงเพศชาย
    male_keywords = ['พ่อ','บิดา','พี่ชาย','น้องชาย','ลูกชาย','สามี','พัว','ผัว','ปู่','ตา','คุณปู่','คุณตา','คุณพ่อ',
                     'ปู่ทวด','ตาทวด','ลุง','อาหนู','คุณอา','คุณลุง','หลายชาย','ลูกเขย','เขย','พี่เขย','น้องเขย',
                     'พ่อตา','พ่อผม','พ่อหนู','พ่อพม','ชาย','หนุ่ม']
    # คำที่บ่งบอกถึงเพศหญิง
    female_keywords = ['แม่','เเม่','คุณแม่','มารดา','พี่สาว','น้องสาว','ลูกสาว','ภรรยา','เมีย','ย่า','ยาย','คุณย่า','คุณยาย','คุณเเม่','ย่าทวด','ยายทวด','ป้า',
                       'น้า','คุณป้า','คุณน้า','หลายสาว','ลูกสะใถ้','ลูกสะใภ้','สะใภ้','พี่สะใภ้','น้องสะใภ้','เเม่ผม','เเม่หนู','เเม่พม','แม่ผม','แม่หนู','แม่พม','สาว','หญิง']
    # ตรวจสอบคำในความคิดเห็น
    for keyword in male_keywords:
        if keyword in comment and "อาการ" not in comment :
              return "ชาย"
    for keyword in female_keywords:
        if keyword in comment:
            return "หญิง"
    # หากไม่พบคำที่บ่งบอกถึงเพศ
    return "ไม่ระบุเพศ"

In [76]:
def detect_gender_self(comment):
    # คำที่บ่งบอกถึงเพศชาย
    male_keywords = ['ผมเป็น','ครับ','ผมเอง','คับ','กระผม','พมเป็น','พมเอง','กระพม']
    # คำที่บ่งบอกถึงเพศหญิง
    female_keywords = ['ค่ะ','คะ','หนูเอง','หนูเป็น','ดิฉัน','ก้อน','คลำ']
    # ตรวจสอบคำในความคิดเห็น
    for keyword in male_keywords:
        if keyword in comment:
            return "เพศชาย"
    for keyword in female_keywords:
        if keyword in comment:
            return "เพศหญิง"
    # หากไม่พบคำที่บ่งบอกถึงเพศ
    return "ไม่ระบุเพศ"

In [77]:
k1=[]
k2=[]
for i in comment['คำพูดโรค']:
  k1.append(detect_person(i))
  if detect_person(i) == 'เล่าประสบการณ์คนอื่น':
    k2.append(detect_gender_other(i))
  elif detect_person(i) == 'เล่าประสบการณ์ตัวเอง':
    k2.append(detect_gender_self(i))
  elif detect_person(i) == 'ไม่ได้เล่าประสบการณ์':
    k2.append(detect_gender_self(i))
r['ใครเล่า'] = k1
r['เพศเเบ่งโดยใช้_python'] = k2

In [78]:
r['เพศเเบ่งโดยใช้_nlp'] = new_colgenden
r['โรค'] = new_colcan

In [79]:
r

,ชื่อ,คำพูดโรค,โรค,คะเเนน,ผลลัพธ์,ใครเล่า,เพศเเบ่งโดยใช้_python,เพศเเบ่งโดยใช้_nlp
0,Akekapong Bowling Singharas,วิธีตรวจมะเร็ง ได้ที่ไหนบ้างครับนอกจาก รพ เจาะ...,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,18,ไม่มีประโยชน์,ไม่ได้เล่าประสบการณ์,ชาย,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น
1,Aliis Chattarika,Theboom Moungna,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,3,ไม่มีประโยชน์,ไม่ได้เล่าประสบการณ์,ไม่ระบุเพศ,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น
2,Alisa Chuasa,Jurairat Norasri. พงษ์นิวัฒน์ ชูอาษา,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,12,ไม่มีประโยชน์,ไม่ได้เล่าประสบการณ์,ไม่ระบุเพศ,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น
3,Amornrat Pollamuangdee,คลำเจอก้อนที่เต้านมค่ะ,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,6,ไม่มีประโยชน์,เล่าประสบการณ์ตัวเอง,หญิง,เพศหญิง
4,Anothai Hassabut,คัดกรองเบื้องต้นด้วยการตรวจเต้านมด้วยตัวเองว่า...,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,12,ไม่มีประโยชน์,เล่าประสบการณ์ตัวเอง,หญิง,เพศหญิง
...,...,...,...,...,...,...,...,...
276,แม่รุ้ง น้องเกล,ลูกสาวมีอาการปวดท้องแบบแน่นๆ เวียนหัว มีจุดจ้ำ...,มะเร็งเม็ดเลือดขาว,40,อาจมีประโยชน์,ไม่ได้เล่าประสบการณ์,ไม่ระบุเพศ,เพศหญิง
277,แอนนี่ น้องข้าวโอ๊ต,Wanitchaya Puruean,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,3,ไม่มีประโยชน์,ไม่ได้เล่าประสบการณ์,ไม่ระบุเพศ,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น
278,แอม' มม มมมมมมม.,มีก้อนเนื้อที่เต้านม ไม่มีอาการเจ็บ แต่คลำๆแล้...,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,29,อาจมีประโยชน์,เล่าประสบการณ์ตัวเอง,หญิง,เพศหญิง
279,โบว์ คะ,จับนมแล้วเจอก้อน,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,5,ไม่มีประโยชน์,เล่าประสบการณ์ตัวเอง,หญิง,เพศหญิง


#sentence_transformers and clustering เเบ่งว่าอันไหนมี ประโยชน์กับไม่มีประโยชน์

In [161]:
from sentence_transformers import SentenceTransformer,util
sentences = comment['คำพูดโรค']
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
embeddings = model.encode(sentences)
#Normalize the embeddings to unit length
Normalize_embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [162]:
clustering_model = KMeans(n_clusters=2)
clustering_model.fit(Normalize_embeddings)
cluster_assignment = clustering_model.labels_
print(cluster_assignment)
print(len(cluster_assignment))

[1 0 0 1 1 0 0 1 1 1 1 0 1 0 0 1 0 0 1 1 0 1 0 0 1 0 0 1 1 0 1 1 1 1 1 1 1
 1 0 0 1 0 1 1 1 1 0 1 0 1 1 1 1 0 0 0 1 0 0 0 0 1 0 0 0 1 1 0 0 1 1 0 0 1
 1 1 1 0 0 0 1 0 0 1 0 0 1 1 0 1 1 1 0 0 0 0 1 1 0 1 0 0 1 1 0 0 0 1 0 0 1
 0 0 0 0 1 0 0 1 1 0 0 1 1 0 0 0 1 0 0 1 1 0 1 0 0 1 0 0 0 1 0 0 0 1 0 1 1
 1 1 1 0 1 0 1 0 1 1 1 0 1 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 1 1 0 1 0 1
 1 1 1 0 1 0 0 0 0 1 0 1 1 0 1 0 1 0 1 1 1 1 1 0 1 1 1 1 0 1 1 1 0 0 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 0 0 1 1 1 1 1 0 1
 1 1 1 1 0 1 0 1 1 1 1 1 1 1 0 0 1 1 0 1 1 1]
281


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [163]:
clusternd_sentences= {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
  if cluster_id not in clusternd_sentences:
    clusternd_sentences[cluster_id] = []

  clusternd_sentences[cluster_id].append(sentences[sentence_id])

In [164]:
len(clusternd_sentences)

2

In [165]:
clusternd_sentences[1][0]

'วิธีตรวจมะเร็ง ได้ที่ไหนบ้างครับนอกจาก รพ เจาะเลือดไปตรวจได้ไหมครับ'

In [166]:
len_1=[]
len_2=[]
for i in range(len(clusternd_sentences)):
    x=len(clusternd_sentences[1][i])
    len_1.append(x)
    y=len(clusternd_sentences[0][i])
    len_2.append(y)
if max(len_1) < max(len_2) :
  Pop=comment[comment['คำพูดโรค'].isin(clusternd_sentences[1])]
  useful = clusternd_sentences[0]
else :
  Pop=comment[comment['คำพูดโรค'].isin(clusternd_sentences[0])]
  useful = clusternd_sentences[1]

In [167]:
Pop

,ชื่อ,คำพูดโรค,โรค,คะเเนน,ผลลัพธ์,ใครเล่า,เพศเเบ่งโดยใช้_python,เพศเเบ่งโดยใช้_nlp
1,Aliis Chattarika,Theboom Moungna,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,3,ไม่มีประโยชน์,ไม่ได้เล่าประสบการณ์,ไม่ระบุเพศ,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น
2,Alisa Chuasa,Jurairat Norasri. พงษ์นิวัฒน์ ชูอาษา,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,12,ไม่มีประโยชน์,ไม่ได้เล่าประสบการณ์,ไม่ระบุเพศ,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น
5,Antina Thanakijareekul,ปอ สรรสวัสดิ์,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,4,ไม่มีประโยชน์,ไม่ได้เล่าประสบการณ์,ไม่ระบุเพศ,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น
6,Aom Phijittra,Preephawat Phawatpokhin,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,3,ไม่มีประโยชน์,ไม่ได้เล่าประสบการณ์,ไม่ระบุเพศ,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น
11,Arisa Sangrodrat,Pakphoom Chaipinidมาอ่าน,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,5,ไม่มีประโยชน์,ไม่ได้เล่าประสบการณ์,ไม่ระบุเพศ,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น
...,...,...,...,...,...,...,...,...
263,สุภารัตน์ เลขาโชค,ไอแห้งๆๆค่ะไอกลางคึน,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,7,ไม่มีประโยชน์,ไม่ได้เล่าประสบการณ์,หญิง,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น
265,สโรชา โศจิรัตน์,ตน,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,1,ไม่มีประโยชน์,ไม่ได้เล่าประสบการณ์,ไม่ระบุเพศ,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น
273,เจ้น๊อต ป๋าแอ๊ด ผัดไทยถาดหาดทรายรี,.,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,1,ไม่มีประโยชน์,ไม่ได้เล่าประสบการณ์,ไม่ระบุเพศ,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น
274,เนเวอร์ เนเวอร์ดาย,อา โนลด์,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,4,ไม่มีประโยชน์,ไม่ได้เล่าประสบการณ์,ไม่ระบุเพศ,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น


In [168]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
useful_embeddings = model.encode(useful)
#Normalize the embeddings to unit length
Normalize_useful_embeddings = useful_embeddings / np.linalg.norm(useful_embeddings, axis=1, keepdims=True)
clustering_useful_model = KMeans(n_clusters=len(name_cancar))
clustering_useful_model.fit(Normalize_useful_embeddings)
cluster_assignment_useful = clustering_useful_model.labels_

print(cluster_assignment_useful)
print(len(cluster_assignment_useful))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[30 18 18  0 33  5 30  7  1  8 26 20 13 18 16 16 27 17 37 30 36  9  9  3
  5 15 11 15 16 10 32 27 13 18 37 26 23 16 21 24 16  1 28  4 12 34  3 27
 13 11  8 26 10  1  1 17 14 18  4  8 37 23 33 19 10 13 16  8 16  7 23 30
 26 23 10  8  2  0 16  8 27  2 30  8  2 33 10  0 30 27  4 10 10  3 14 30
  8 14  6  7  8 37 25 13 26 29 23 35 27 16 26  5 24 13 10  9 31 30 30  1
 20 30 30 22 14 27 37 15  2 23 20  2  6 12 36 27 17  6 23 33  2 10 10  1
  1 19 33 20  6 31 11  6 16 15 19  8 14 20 23  9 24 10]
162


In [169]:
type(useful)

list

In [170]:
clusternd_useful_sentences= {}
for sentence_ID, cluster_ID in enumerate(cluster_assignment_useful):
  if cluster_ID not in clusternd_useful_sentences:
    clusternd_useful_sentences[cluster_ID] = []

  clusternd_useful_sentences[cluster_ID].append(useful[sentence_ID])

In [205]:
comment[comment['คำพูดโรค'].isin(clusternd_useful_sentences[7])]

,ชื่อ,คำพูดโรค,โรค,คะเเนน,ผลลัพธ์,ใครเล่า,เพศเเบ่งโดยใช้_python,เพศเเบ่งโดยใช้_nlp
12,Arita Barameeanant,ท่านใดพอทราบไหมคะว่าเราสามารถทำประกันอะไรไว้ได...,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,26,อาจมีประโยชน์,ไม่ได้เล่าประสบการณ์,หญิง,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น
146,Santiparp Subgampang,เริ่มจากมีตังก่อน เพราะเดี๋ยวประกันจะตามมาเสนอ...,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,14,ไม่มีประโยชน์,เล่าประสบการณ์คนอื่น,ชาย,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น
204,คุณชาย ทานตะวัน,แบบไหน หรือไม่แบบไหนก็ตรวจไปก่อน ถ้าตังไม่ถึงก...,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,19,ไม่มีประโยชน์,ไม่ได้เล่าประสบการณ์,ไม่ระบุเพศ,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น


In [212]:
def define_Cencer_with_clusternd(use_clusternd_sentences,n_clusters):
  data = pd.DataFrame()
  for i in range(n_clusters):
    point = []
    define_C =[]
    test = comment[comment['คำพูดโรค'].isin(use_clusternd_sentences[i])]
    test_cer = list(test['โรค'])
    for j in range(len(test_cer)):
      if test_cer[j] in name_cancar:
        point.append(test_cer[j])
    if len(set(point)) == 0 :
      for x in range(len(test_cer)):
        define_C.append('ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น')
    elif len(set(point)) == 1 :
      for x in range(len(test_cer)):
        define_C.append(point[0])
    elif len(set(point)) > 1:
      for x in range(len(test_cer)):
        define_C.append('มีโอกาสเป็นโรคมะเร็งมากกว่า 2 เเบบ')
    test['โรค_clusternd'] = define_C
    data = pd.concat([data,test])
  return data

In [213]:
a=define_Cencer_with_clusternd(clusternd_useful_sentences,len(name_cancar))

<ipython-input-212-a6fb8fd19b3f>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['โรค_clusternd'] = define_C
<ipython-input-212-a6fb8fd19b3f>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['โรค_clusternd'] = define_C
<ipython-input-212-a6fb8fd19b3f>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

In [214]:
a

,ชื่อ,คำพูดโรค,โรค,คะเเนน,ผลลัพธ์,ใครเล่า,เพศเเบ่งโดยใช้_python,เพศเเบ่งโดยใช้_nlp,โรค_clusternd
7,Aommy Janjira Charenying,มะเร็งแต่ละอาการไม่เหมือนกันค่ะ แต่มีเลือดออกม...,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,13,ไม่มีประโยชน์,ไม่ได้เล่าประสบการณ์,หญิง,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น
157,Su Jira,มะเร็งมันไม่ใช่เป็นปุ๊บปับ มันใช้เวลาเพาะบ่ม น...,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,19,ไม่มีประโยชน์,ไม่ได้เล่าประสบการณ์,ไม่ระบุเพศ,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น
182,Verasu Matroj,ก้อนไขมัน กับ ก้อนมะเร็ง ต่างกันมากมั้ยครับ,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,12,ไม่มีประโยชน์,เล่าประสบการณ์ตัวเอง,ชาย,เพศหญิง,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น
15,Asisah Kasor,มีก้อนที่เต้านมคะหลังจากเจอก็ไปหาหมอทันทีตอนแร...,มะเร็งเต้านม,49,อาจมีประโยชน์,เล่าประสบการณ์ตัวเอง,หญิง,เพศหญิง,มะเร็งเต้านม
75,Marco Po Lo,ปวดใต้รักแร้ค่ะประมาณหนึ่งอาทิตย์เริ่มจับหน้าอ...,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,58,มีประโยชน์,เล่าประสบการณ์คนอื่น,หญิง,เพศหญิง,มะเร็งเต้านม
...,...,...,...,...,...,...,...,...,...
33,Copter Achirathon Jaising,เท่าที่อ่านมาสามหมื่นแปดพันอาการ,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,8,ไม่มีประโยชน์,ไม่ได้เล่าประสบการณ์,ไม่ระบุเพศ,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น
61,Kamonporn Tengpawadee,อุจาระเปลี่ยนสี ไม่มีแรง ไม่มีเสียงไม่ต้องไปตร...,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,22,ไม่มีประโยชน์,ไม่ได้เล่าประสบการณ์,ไม่ระบุเพศ,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น
122,Piizchy Panwalee,แรกๆเป้นทุกอย่าง ทั้งคันตามตัว เหงื่อออก เหนื่...,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,26,อาจมีประโยชน์,เล่าประสบการณ์คนอื่น,ชาย,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น
206,ฅน ผาปืนแตก,ไอเรื้อรัง เหนื่อยง่าย มีไข้ทุกวัน\r\nเจ็บหน้า...,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,31,อาจมีประโยชน์,เล่าประสบการณ์คนอื่น,หญิง,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น
